In [ ]:
%pip install -U scikit-fuzzy
%pip install numpy
%pip install pandas
# o scikit-fuzzy depende da lib abaixo
%pip install networkx


Aplicando logica fuzzy na atividade dos municipios

In [ ]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

### Significado das colunas (variaveis de entrada) no dataset
* 'perc_dom_pobres_var1'    => Percentual entre quantidade de domicílios pobres e o número de domicílios no município
* 'perc_pobres_0a11_var2'   => Percentual de crianças pobres entre 0 e 11 anos
* 'per_dom_vuln_var3'       => Percentual entre quantidade de domicílios vulneráveis (em risco) pobres e o número de domicílios no município
* 'num_idhm'                => IDH do município

### Significado das colunas (variaveis de entrada) no dataset
* 'necessidade_auxilio'    => Indicador de Necessidade de Auxílio Governamental Urgente para a Infância

In [ ]:
# Caminho do arquivo carregado
dataset_path = './MunicipioBrasil_20230102.csv'

# Lendo o arquivo para capturar colunas úteis
selected_columns_fuzzy = ['perc_dom_pobres_var1', 'perc_pobres_0a11_var2', 'per_dom_vuln_var3', 'num_idhm']
selected_columns_general = ['cod_mun', 'nom_mun']
selected_columns = selected_columns_fuzzy + selected_columns_general

df = pd.read_csv(dataset_path, usecols=selected_columns)

# Convertendo as colunas de string para float (substituindo vírgulas por pontos)
for column in selected_columns_fuzzy:
    df[column] = df[column].str.replace(',', '.').astype(float)

# Substituindo valores NaN por 0 em todo o DataFrame
df = df.fillna(0)

df.head(5)

In [ ]:
# Criando a variável fuzzy de SAIDA
## referente ao Indicador de Necessidade de Auxílio Governamental Urgente para a Infância
necessidade_auxilio     = ctrl.Consequent(np.arange(0, 101, 1), 'necessidade_auxilio') 

In [ ]:
# Calculando os universos fuzzy (limites) das variaveis de ENTRADA com base nos valores das colunas
domicilio_pobre_range       = (df['perc_dom_pobres_var1'].min()     , df['perc_dom_pobres_var1'].max())
crianca_pobre_range         = (df['perc_pobres_0a11_var2'].min()    , df['perc_pobres_0a11_var2'].max())
domicilio_vulneravel_range  = (df['per_dom_vuln_var3'].min()        , df['per_dom_vuln_var3'].max())
idh_municipio_range         = (df['num_idhm'].min()                 , df['num_idhm'].max())

# Criando as variáveis fuzzy de ENTRADA
## de entrada com base nos valores do DataFrame
domicilio_pobre         = ctrl.Antecedent(np.linspace(domicilio_pobre_range[0]          , domicilio_pobre_range[1]      , 100), 'domicilio_pobre')
crianca_pobre           = ctrl.Antecedent(np.linspace(crianca_pobre_range[0]            , crianca_pobre_range[1]        , 100), 'crianca_pobre')
domicilio_vulneravel    = ctrl.Antecedent(np.linspace(domicilio_vulneravel_range[0]     , domicilio_vulneravel_range[1] , 100), 'domicilio_vulneravel')
idh_municipio           = ctrl.Antecedent(np.linspace(idh_municipio_range[0]            , idh_municipio_range[1]        , 100), 'idh_municipio')

In [ ]:
# Definindo os termos linguísticos para as variáveis fuzzy de SAIDA
necessidade_auxilio['baixissimo']   = fuzz.trapmf(necessidade_auxilio.universe, [0, 0, 5, 10])
necessidade_auxilio['baixo']        = fuzz.trimf(necessidade_auxilio.universe, [6, 10, 30])
necessidade_auxilio['mediano']      = fuzz.trimf(necessidade_auxilio.universe, [11, 30, 70])
necessidade_auxilio['alto']         = fuzz.trimf(necessidade_auxilio.universe, [31, 70, 100])
necessidade_auxilio['altissimo']    = fuzz.trapmf(necessidade_auxilio.universe, [71, 100, 100, 100])

# Definindo os termos linguísticos para as variáveis fuzzy de ENTRADA
## Variáveis de entrada
domicilio_pobre['baixissimo']   = fuzz.trapmf(domicilio_pobre.universe, [0, 0, 20, 40])
domicilio_pobre['baixo']        = fuzz.trimf(domicilio_pobre.universe, [21, 40, 60])
domicilio_pobre['mediano']      = fuzz.trimf(domicilio_pobre.universe, [41, 60, 70])
domicilio_pobre['alto']         = fuzz.trimf(domicilio_pobre.universe, [61, 70, 100])
domicilio_pobre['altissimo']    = fuzz.trapmf(domicilio_pobre.universe, [71, 100, 100, 100])

idh_municipio['baixo']          = fuzz.trapmf(idh_municipio.universe, [0, 0, 0.499, 0.5])
idh_municipio['medio']          = fuzz.trimf(idh_municipio.universe, [0.5, 0.799, 0.8])
idh_municipio['alto']           = fuzz.trapmf(idh_municipio.universe, [0.8, 1, 1, 1])

domicilio_vulneravel['baixissimo']  = fuzz.trapmf(domicilio_vulneravel.universe, [0, 0, 5, 10])
domicilio_vulneravel['baixo']       = fuzz.trimf(domicilio_vulneravel.universe, [6, 10, 30])
domicilio_vulneravel['mediano']     = fuzz.trimf(domicilio_vulneravel.universe, [11, 30, 70])
domicilio_vulneravel['alto']        = fuzz.trimf(domicilio_vulneravel.universe, [31, 70, 100])
domicilio_vulneravel['altissimo']   = fuzz.trapmf(domicilio_vulneravel.universe, [71, 100, 100, 100])

crianca_pobre['baixissimo']     = fuzz.trapmf(crianca_pobre.universe, [0, 0, 2, 10])
crianca_pobre['baixo']          = fuzz.trimf(crianca_pobre.universe, [3, 10, 20])
crianca_pobre['mediano']        = fuzz.trimf(crianca_pobre.universe, [11, 20, 35])
crianca_pobre['alto']           = fuzz.trimf(crianca_pobre.universe, [20, 35, 100])
crianca_pobre['altissimo']      = fuzz.trapmf(crianca_pobre.universe, [36, 100, 100, 100])

In [ ]:
# Regras fuzzy com no máximo 2 a 3 variáveis de entrada, limitado a 2 regras por saída
rules = [
    # Regras para "baixíssimo"
    ctrl.Rule(domicilio_pobre['baixissimo'] & crianca_pobre['baixissimo'], necessidade_auxilio['baixissimo']),
    ctrl.Rule(idh_municipio['alto'] & crianca_pobre['baixissimo'], necessidade_auxilio['baixissimo']),

    # Regras para "baixo"
    ctrl.Rule(domicilio_pobre['baixo'] & crianca_pobre['baixo'], necessidade_auxilio['baixo']),
    ctrl.Rule(idh_municipio['medio'] & domicilio_vulneravel['baixo'], necessidade_auxilio['baixo']),

    # Regras para "mediano"
    ctrl.Rule(domicilio_pobre['mediano'] & crianca_pobre['mediano'], necessidade_auxilio['mediano']),
    ctrl.Rule(idh_municipio['medio'] & domicilio_vulneravel['mediano'], necessidade_auxilio['mediano']),

    # Regras para "alto"
    ctrl.Rule(domicilio_pobre['alto'] & crianca_pobre['alto'], necessidade_auxilio['alto']),
    ctrl.Rule(domicilio_pobre['alto'] & crianca_pobre['altissimo'], necessidade_auxilio['alto']),
    ctrl.Rule(domicilio_pobre['altissimo'] & crianca_pobre['alto'], necessidade_auxilio['alto']),
    ctrl.Rule(idh_municipio['baixo'] & domicilio_vulneravel['alto'], necessidade_auxilio['alto']),
    ctrl.Rule(idh_municipio['medio'] & crianca_pobre['alto'], necessidade_auxilio['alto']),

    # Regras para "altíssimo"
    ctrl.Rule(domicilio_pobre['altissimo'] & crianca_pobre['altissimo'], necessidade_auxilio['altissimo']),
    ctrl.Rule(idh_municipio['baixo'] & domicilio_vulneravel['altissimo'], necessidade_auxilio['altissimo']),
    ctrl.Rule(idh_municipio['baixo'] & domicilio_pobre['altissimo'], necessidade_auxilio['altissimo']),
]

# Criar o sistema de controle fuzzy
necessidade_auxilio_ctrl = ctrl.ControlSystem(rules)
necessidade_auxilio_sim = ctrl.ControlSystemSimulation(necessidade_auxilio_ctrl)


In [ ]:
# Define a função para categorizar os valores
def categorizar(valor):

    if 0 <= valor <= 5:
        return "baixissimo"
    elif 6 <= valor <= 10:
        return "baixo"
    elif 11 <= valor <= 30:
        return "mediano"
    elif 31 <= valor <= 70:
        return "alto"
    elif 71 <= valor <= 100:
        return "altissimo"
    else:
        return "fora do intervalo"


def calcula_auxilio(row):
    global necessidade_auxilio_sim

    necessidade_auxilio_sim.input['domicilio_pobre']        = row['perc_dom_pobres_var1']
    necessidade_auxilio_sim.input['idh_municipio']          = row['num_idhm']
    necessidade_auxilio_sim.input['domicilio_vulneravel']   = row['per_dom_vuln_var3']
    necessidade_auxilio_sim.input['crianca_pobre']          = row['perc_pobres_0a11_var2']
    necessidade_auxilio_sim.compute()
    try:
        resultado = necessidade_auxilio_sim.output['necessidade_auxilio']
    except:
        resultado = 0

    return resultado

df['necessidade_auxilio'] = -1
df['necessidade_auxilio'] = df.apply(calcula_auxilio, axis=1)
df["resultado"] = df["necessidade_auxilio"].apply(categorizar)
df.to_csv('./resultado_potencial_auxilio_municipios_simfuzzy.csv', index=False)

# printa resumo do resultado
print(df['resultado'].value_counts())